In [1]:
import json
import requests
from datetime import datetime, timedelta
import pickle
import xgboost
import pandas as pd

In [2]:
object_id_dict = {
    255439: "MP 01: Naamsestraat 35 Maxim",
    255440: "MP 02: Naamsestraat 57 Xior",
    255441: "MP 03: Naamsestraat 62 Taste",
    255442: "MP 05: Calvariekapel KU Leuven",
    255443: "MP 06: Parkstraat 2 La Filosovia",
    255444: "MP 07: Naamsestraat 81",
    280324: "MP08bis - Vrijthof",
}

In [3]:
def get_forecast_hourly_weather():
    url = "https://api.open-meteo.com/v1/forecast?latitude=50.88&longitude=4.70&hourly=temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,rain,showers,snowfall,weathercode,pressure_msl,surface_pressure,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,evapotranspiration,et0_fao_evapotranspiration,vapor_pressure_deficit,windspeed_10m,winddirection_10m,windgusts_10m,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance&models=best_match&timezone=Europe%2FBerlin"
    resp = requests.get(url)
    data = resp.json()
    df = pd.DataFrame(data["hourly"])
    return df


def get_forecast_hourly_air():
    url = "https://air-quality-api.open-meteo.com/v1/air-quality?latitude=50.88&longitude=4.70&hourly=pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,ozone,ammonia&timezone=Europe%2FBerlin"
    resp = requests.get(url)
    data = resp.json()
    df = pd.DataFrame(data["hourly"])
    return df

In [4]:
# Get 4 days
today = datetime.now().date()
next_4_days = [today + timedelta(days=i) for i in range(4)]

In [5]:
# Get weather
hourly_weather = get_forecast_hourly_weather()
hourly_weather["time"] = pd.to_datetime(hourly_weather["time"])
hourly_weather = hourly_weather[hourly_weather["time"].dt.date.isin(next_4_days)]
# selected_hourly_weather = hourly_weather[hourly_weather["time"].dt.date == selected_date]
hourly_weather["precipitation"] = (
    hourly_weather["rain"] + hourly_weather["showers"] + hourly_weather["snowfall"]
)
hourly_weather.drop(
    [
        "showers",
        "et0_fao_evapotranspiration",
        "evapotranspiration",
        "vapor_pressure_deficit",
    ],
    axis=1,
    inplace=True,
)

# Get air
hourly_air = get_forecast_hourly_air()
hourly_air["time"] = pd.to_datetime(hourly_air["time"])
hourly_air = hourly_air[hourly_air["time"].dt.date.isin(next_4_days)]
# selected_hourly_air = hourly_air[hourly_air["time"].dt.date == selected_date]
hourly_air.rename(
    columns={
        "carbon_monoxide": "co",
        "nitrogen_dioxide": "no2",
        "sulphur_dioxide": "so2",
        "ozone": "o3",
        "ammonia": "nh3",
    },
    inplace=True,
)


# Merge
hourly_df = pd.merge(hourly_weather, hourly_air, on="time")
hourly_df["date"] = hourly_df["time"].dt.date
hourly_df["hour"] = hourly_df["time"].dt.hour
hourly_df["month"] = hourly_df["time"].dt.month
hourly_df["weekday"] = hourly_df["time"].dt.strftime("%a")
hourly_df.drop(["time"], axis=1, inplace=True)
hourly_df.head(3)

,temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,rain,snowfall,weathercode,pressure_msl,surface_pressure,cloudcover,...,pm2_5,co,no2,so2,o3,nh3,date,hour,month,weekday
0,15.5,39,1.6,12.4,0.0,0.0,0,1020.7,1017.6,0,...,4.9,127.0,7.6,1.5,72.0,5.0,2023-06-04,0,6,Sun
1,14.4,44,2.3,11.7,0.0,0.0,0,1020.9,1017.8,0,...,4.9,126.0,9.2,1.4,66.0,5.5,2023-06-04,1,6,Sun
2,13.3,50,3.1,10.7,0.0,0.0,0,1021.3,1018.1,0,...,5.6,131.0,10.0,1.4,62.0,5.5,2023-06-04,2,6,Sun


In [6]:
def add_object_id(hourly_df, object_ids):
    dfs = []
    for id in object_ids:
        new_df = hourly_df.copy()
        new_df["object_id"] = id
        dfs.append(new_df)

    combined_df = pd.concat(dfs)
    combined_df.reset_index(drop=True, inplace=True)
    return combined_df

In [7]:
with open("../model/model_noise_level_file42/encoder_model_file42.pkl", "rb") as file:
    encoder = pickle.load(file)

In [8]:
with open("../model/model_noise_level_file42/xgb_laeq.pkl", "rb") as file:
    laeq_model = pickle.load(file)

In [9]:
object_ids = list(object_id_dict.keys())
df = add_object_id(hourly_df, object_ids)
transformed_df = encoder.transform(df)
laeq_pred = laeq_model.predict(transformed_df)

In [10]:
laeq_pred[:10]

array([47.20945 , 44.51925 , 41.635227, 44.954826, 42.161335, 42.53615 ,
       41.79713 , 45.8285  , 45.335297, 47.114197], dtype=float32)